In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install nltk bert-score rouge-score 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6f06d526bd90f9498ffdae8edc1ddaba7e8bb0c7bb533df360a4fdd1d1074566
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: rich
    Found existing installation: rich 13.7.1
    Uninstalling rich-13.7.1:
      Successfully uninstalled rich-13.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
import torch
import warnings
import pandas as pd
import time
import re
from dotenv import load_dotenv
import os
from together import Together
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from datasets import load_dataset
warnings.filterwarnings('ignore')

In [4]:
from tqdm import tqdm

In [7]:
df_explanation = pd.read_csv('/kaggle/input/explanations-new/explanations (1).csv')

def remove_jpg_extension(image_name):
    final_image_name = image_name.split('.')[0]
    return final_image_name

df_explanation['image'] = df_explanation['image'].apply(remove_jpg_extension)

df_explanation.drop('Unnamed: 2',axis=1,inplace=True)

In [9]:
df_test = pd.read_csv('/kaggle/input/test-tsv1/test_df.tsv',sep='\t')
df_test.columns = ["image",'captions','sarcasm_explanations']
df_test

,image,captions,sarcasm_explanations
0,700183392969756672,'oh i so love working late from home # work #...,the author hates working late from home.
1,928753954745475072,'yeaah ! buddy o miracle worker # infj emoji_...,"your anxiety is not cured when someone says ""d..."
2,935133439011049473,'rt <user> : something different ..... a delay...,the author is pissed to watch a full train lea...
3,933466049697198080,'oh really linkedin ? thanks for the super use...,the author doesn't find such notifications fro...
4,699788403940978689,'our beautiful view along pch ! # pch # highw...,the author can't really see the view because o...
...,...,...,...
346,1011850445043480900_1580447253,âï¸âï¸âï¸âï¸âï¸âï¸âï¸âï...,it isn't a cool week if it's 100 degrees.
347,636237012294327831_256939246,She's nothing like her dad... ;) #lovethem #fa...,"she's exactly like her dad, both are making si..."
348,899685897251069952,'this eclipse is even cooler than i thought it...,the author is disappointed with this eclipse s...
349,878368201221914624,'<user> app radar is definitely right on targe...,<user> app radar isn't right on target.


In [11]:
test_image_list = df_test['image'].to_list()
df_explanation = df_explanation[df_explanation['image'].isin(test_image_list)]

In [12]:
df_explanation

,image,llm_explanation
0,899095756979576832,The image depicts a large outdoor screen broad...
2,704299596550434816,The image shows a message from an unknown poli...
3,1030084668871077796_1449452234,The image showcases a humorous quote about a p...
17,858271535584468992,The image presents a concise yet informative v...
28,725109055434289152,The image shows a weather forecast for Pittsbu...
...,...,...
3457,899685897251069952,The image depicts a photograph of a blue sky w...
3470,566063947261064369_324307476,The image shows a white car parked in a handic...
3482,869958145790980097,The image is a notice announcing the permanent...
3494,1000900004679480792_43891756,The image presents a close-up view of a broken...


In [13]:
final_df = pd.merge(df_explanation,df_test,how='inner',on='image')
final_df

,image,llm_explanation,captions,sarcasm_explanations
0,899095756979576832,The image depicts a large outdoor screen broad...,'seriously why would you ever live here ... #...,"who wouldn't live here, it's fun."
1,704299596550434816,The image shows a message from an unknown poli...,'awww poor cruz is broke and desperately needs...,this mail is fraud and one shouldn’t really be...
2,1030084668871077796_1449452234,The image showcases a humorous quote about a p...,HAPPENS EVERY TIME @rebelcircus #rebelcircus #...,it's awful to have to pee after settling into ...
3,858271535584468992,The image presents a concise yet informative v...,'# brexit explained see how the eu keeps all t...,the eu doesn't keep the money for itself.
4,725109055434289152,The image shows a weather forecast for Pittsbu...,forecast for pittsburgh is looking great ! can...,forecast for pittsburgh is rainy and not great...
...,...,...,...,...
346,899685897251069952,The image depicts a photograph of a blue sky w...,'this eclipse is even cooler than i thought it...,the author is disappointed with this eclipse s...
347,566063947261064369_324307476,The image shows a white car parked in a handic...,This guy gets a gold star for excellent parking.,this guy has parked his car partially covering...
348,869958145790980097,The image is a notice announcing the permanent...,'thanks to our elected mla and mp for raising ...,the author is pissed at their elected mla and ...
349,1000900004679480792_43891756,The image presents a close-up view of a broken...,Wow thanks Apple! I've never had a cord last t...,one hates when apple cords don't last longer t...


In [14]:
final_df['captions'][0]

"'seriously why would you ever live here ...  # lastbestplace # gocatsgo'"

In [15]:
context1 = f"This is the caption of image {final_df['captions'][0]}. The literal explanations of image is {final_df['llm_explanation'][0]}. The sarcasm explanation of the image given the caption is {final_df['sarcasm_explanations'][0]}."
context2 = f"This is the caption of image {final_df['captions'][1]}. The literal explanations of image is {final_df['llm_explanation'][1]}. The sarcasm explanation of the image given the caption is {final_df['sarcasm_explanations'][1]}."
context3 = f"This is the caption of image {final_df['captions'][1]}. The literal explanations of image is {final_df['llm_explanation'][1]}. The sarcasm explanation of the image given the caption is {final_df['sarcasm_explanations'][1]}."

In [16]:
import os
from together import Together

os.environ['TOGETHER_API_KEY'] = 'dc2960d78c59037c21bfc5f40e79b2597d9a37d856e504bdbb393a99702984aa'

api_key = os.environ.get('TOGETHER_API_KEY')

if not api_key:
    raise ValueError("API key not found. Please set the TOGETHER_API_KEY environment variable.")

client = Together(api_key=api_key)

In [17]:
generated_explanations = []

for index, row in tqdm(final_df.iterrows(), total=len(final_df), dynamic_ncols=True):
    caption = row['captions']
    exp = row['llm_explanation']
    
    # Few-shot learning prompt construction with examples
    prompt = f"""
    Here are a few examples:
    
    {context1}
    {context2}
    
    Now, given this new image caption: {caption}, and this explanation: {exp}, explain the hidden sarcasm in just one sentence.
    """
    
    try:
        response = client.chat.completions.create(
            model="Qwen/Qwen2.5-72B-Instruct-Turbo",
            messages=[
                {
                    "role" : "system",
                    "content" : "You are a helpful assistant who explains hidden sarcasm in images based on image explanations and captions."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            max_tokens=512,
            temperature=0.7,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1,
            stop=["<|eot_id|>", "<|eom_id|>"],
            stream=False  
        )
        
        if hasattr(response, 'choices') and response.choices:
            choice = response.choices[0]
            if hasattr(choice, 'message') and choice.message:
                content = choice.message.content
                generated_explanations.append(content)
            else:
                generated_explanations.append("No message content found.")
        else:
            generated_explanations.append("No choices found in the response.")
    
    except Exception as e:
        generated_explanations.append(f"Error: {e}")

100%|██████████| 351/351 [07:16<00:00,  1.24s/it]


In [18]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score

In [19]:
final_df

,image,llm_explanation,captions,sarcasm_explanations
0,899095756979576832,The image depicts a large outdoor screen broad...,'seriously why would you ever live here ... #...,"who wouldn't live here, it's fun."
1,704299596550434816,The image shows a message from an unknown poli...,'awww poor cruz is broke and desperately needs...,this mail is fraud and one shouldn’t really be...
2,1030084668871077796_1449452234,The image showcases a humorous quote about a p...,HAPPENS EVERY TIME @rebelcircus #rebelcircus #...,it's awful to have to pee after settling into ...
3,858271535584468992,The image presents a concise yet informative v...,'# brexit explained see how the eu keeps all t...,the eu doesn't keep the money for itself.
4,725109055434289152,The image shows a weather forecast for Pittsbu...,forecast for pittsburgh is looking great ! can...,forecast for pittsburgh is rainy and not great...
...,...,...,...,...
346,899685897251069952,The image depicts a photograph of a blue sky w...,'this eclipse is even cooler than i thought it...,the author is disappointed with this eclipse s...
347,566063947261064369_324307476,The image shows a white car parked in a handic...,This guy gets a gold star for excellent parking.,this guy has parked his car partially covering...
348,869958145790980097,The image is a notice announcing the permanent...,'thanks to our elected mla and mp for raising ...,the author is pissed at their elected mla and ...
349,1000900004679480792_43891756,The image presents a close-up view of a broken...,Wow thanks Apple! I've never had a cord last t...,one hates when apple cords don't last longer t...


In [20]:
y_true = final_df['sarcasm_explanations'].values.tolist()
y_pred = generated_explanations

bleu_1 = 0
bleu_2 = 0
bleu_3 = 0
bleu_4 = 0
count = 0
weights_1 = (1./1.,)
weights_2 = (1./2. , 1./2.)
weights_3 = (1./3., 1./3., 1./3.)
weights_4 = (1./4., 1./4., 1./4., 1./4.)
met = 0

for reference, hypothesis in zip(y_true, y_pred):
    # met += nltk.translate.meteor_score.meteor_score([reference], hypothesis)
    reference = reference.split()
    hypothesis = hypothesis.split()
    bleu_1 += sentence_bleu([reference], hypothesis, weights_1) 
    bleu_2 += sentence_bleu([reference], hypothesis, weights_2)
    bleu_3 += sentence_bleu([reference], hypothesis, weights_3)
    bleu_4 += sentence_bleu([reference], hypothesis, weights_4)
    count += 1

bleu_1 = bleu_1/count
bleu_2 = bleu_2/count
bleu_3 = bleu_3/count
bleu_4 = bleu_4/count
# met = met/count
print("BLEU-1:", bleu_1)
print("BLEU-2:", bleu_2)
print("BLEU-3:", bleu_3)
print("BLEU-4:", bleu_4)

BLEU-1: 0.11750190832279356
BLEU-2: 0.20049589800328455
BLEU-3: 0.30165589923783664
BLEU-4: 0.38426018755656755


In [21]:
nltk.download('wordnet')

bleu_scores = []

references = final_df['sarcasm_explanations'].values.tolist()
e_new = generated_explanations

for ref, gen in zip(references, e_new):
    reference_tokens = [ref.split()]  
    candidate_tokens = gen.split()  
    score = sentence_bleu(reference_tokens, candidate_tokens)
    bleu_scores.append(score)
    print(f"BLEU score for reference: {ref} \nGenerated: {gen}\nScore: {score:.4f}\n")

average_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score: {average_score:.4f}")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
BLEU score for reference: who wouldn't live here, it's fun. 
Generated: The caption sarcastically questions why anyone would want to live there, when the image clearly shows a fun and relaxing community event, suggesting it's actually a great place to live.
Score: 0.5081

BLEU score for reference: this mail is fraud and one shouldn’t really be worried about it. 
Generated: The caption mockingly suggests that the politician's plea for donations is so exaggerated that it's almost comical, implying that one shouldn't take it seriously.
Score: 0.5373

BLEU score for reference: it's awful to have to pee after settling into bed with blankets, laptop, headphones, a drink and everything else. 
Generated: The sarcasm lies in the contrast between the perfect setup for a cozy night in and the inevitable and inconvenient need to use the restroom, highlighting the universal frustr

In [22]:
from rouge_score import rouge_scorer
import numpy as np

# Initialize the scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Assume `references` is a list of reference texts and `e_new` is the list of generated texts.
      # Your list of generated texts

# Initialize accumulators for ROUGE scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Iterate over reference and generated text pairs
for ref, gen in zip(references, e_new):
    scores = scorer.score(ref, gen)
    
    # Append scores to the accumulators
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)
    
    # Print individual scores (optional)
    print(f"Reference: {ref}\nGenerated: {gen}\n")
    print(f"ROUGE-1: {scores['rouge1']}")
    print(f"ROUGE-2: {scores['rouge2']}")
    print(f"ROUGE-L: {scores['rougeL']}\n")

# Calculate the average scores for each metric
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

# Print the average ROUGE scores
print(f"Average ROUGE-1 F1: {avg_rouge1}")
print(f"Average ROUGE-2 F1: {avg_rouge2}")
print(f"Average ROUGE-L F1: {avg_rougeL}")

Reference: who wouldn't live here, it's fun.
Generated: The caption sarcastically questions why anyone would want to live there, when the image clearly shows a fun and relaxing community event, suggesting it's actually a great place to live.

ROUGE-1: Score(precision=0.12903225806451613, recall=0.5, fmeasure=0.20512820512820512)
ROUGE-2: Score(precision=0.03333333333333333, recall=0.14285714285714285, fmeasure=0.05405405405405406)
ROUGE-L: Score(precision=0.0967741935483871, recall=0.375, fmeasure=0.15384615384615383)

Reference: this mail is fraud and one shouldn’t really be worried about it.
Generated: The caption mockingly suggests that the politician's plea for donations is so exaggerated that it's almost comical, implying that one shouldn't take it seriously.

ROUGE-1: Score(precision=0.18518518518518517, recall=0.38461538461538464, fmeasure=0.25)
ROUGE-2: Score(precision=0.07692307692307693, recall=0.16666666666666666, fmeasure=0.10526315789473684)
ROUGE-L: Score(precision=0.1851

In [23]:
from collections import Counter

def simple_meteor_score(reference, e_new):
    def word_matches(ref, hyp):
        ref_counter = Counter(ref.split())
        hyp_counter = Counter(hyp.split())
        
        matches = sum(min(ref_counter[word], hyp_counter[word]) for word in hyp_counter)
        
        return matches
    
    def precision_recall(matches, reference_len, e_new_len):
        precision = matches / e_new_len if e_new_len > 0 else 0
        
        recall = matches / reference_len if reference_len > 0 else 0
        
        return precision, recall
    
    all_scores = []
    
    for hyp in e_new:
        hypothesis_scores = []
        for ref in reference:
            matches = word_matches(ref, hyp)
            precision, recall = precision_recall(matches, len(ref.split()), len(hyp.split()))

            if precision + recall == 0:
                f_mean = 0
            else:
                f_mean = (10 * precision * recall) / (9 * precision + recall)
            
            hypothesis_scores.append(f_mean)
        all_scores.append(max(hypothesis_scores))
    return np.mean(all_scores)


meteor_score = simple_meteor_score(references, e_new)
print(f"Simplified METEOR score: {meteor_score:.4f}")

Simplified METEOR score: 0.3914
